In [9]:
import os
from pprint import pprint
from langchain.llms import AzureOpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# print environment OPENAI_API_BASE
print(os.environ.get("OPENAI_API_BASE"))

https://alkopenai2.openai.azure.com/


In [11]:
# from env variable MONGO_CONNECTION access collection 
# DocumentSegments from DocSampleV1 database
from pymongo import MongoClient

client = MongoClient(os.environ.get("MONGO_CONNECTION"))
db = client.DocSampleV1
collection = db.DocumentSegments

# print number of documents inside the database
count = collection.count_documents({})
print(f"Found {count} segments in the database")


Found 484 segments in the database


In [24]:
from sentence_transformers import SentenceTransformer

# now load all documents from the collection taking only the 
# property content and put everything in an array of bson document
documents = []
ids = []
for doc in collection.find({}, {"_id": 1, "Content": 1}):
    documents.append(doc["Content"])   
    ids.append(doc["_id"]) 


In [15]:

transformers_cache = os.environ.get('TRANSFORMERS_CACHE')
print(transformers_cache)
model2 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', cache_folder=transformers_cache)


None


In [25]:

embeddings2 = model2.encode(documents)

Now it is the time to represent all the content of the book into a graph

True


In [21]:
# This is copied from 03_vector_search.ipynb
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import umap

import altair as alt
def plot_embeddings(sentences, embeddings):
    panda_sentences = pd.DataFrame({'text':sentences})

    # UMAP reduces dimension to a plottable 2DE
    reducer = umap.UMAP(n_neighbors=2)
    umap_embeds = reducer.fit_transform(embeddings)
    
    # create a dataframe with the umap embeddings and the corresponding sentences
    df_plot = pd.DataFrame({'x': umap_embeds[:, 0], 'y': umap_embeds[:, 1], 'text': panda_sentences['text']})
    
    # create a column to specify the color of each point
    
    # create the interactive scatter plot with labels
    return alt.Chart(df_plot, width=1100, height=600).mark_circle(size=60).encode(
        x='x',
        y='y',
        tooltip=['text']
    )


In [22]:
chart  = plot_embeddings(documents, embeddings2)

chart.interactive()

/Users/gianmariaricci/develop/github/ai-playground/src/python/langchainVarious/langchain/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

Now that I have all the embeddings, I surely need to add the embeddings inside the original database elements

In [32]:
# use pymongo to remove property DenseVectorList abn DenseVector_all-mpnet-base-v2 from all documents
# in the collection DocumentSegments
collection.update_many({}, {"$unset": {"DenseVectorList": "", "DenseVector_all-mpnet-base-v2": ""}})

UpdateResult({'n': 484, 'electionId': ObjectId('7fffffff00000000000002b6'), 'opTime': {'ts': Timestamp(1699206308, 509), 't': 694}, 'nModified': 484, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1699206308, 516), 'signature': {'hash': b'f\xaf1\xd5\xb3\x0eA:\x98t\xd1\xcc\t\xc0\xaeDR\x92\x8a\xa1', 'keyId': 7261999934654644226}}, 'operationTime': Timestamp(1699206308, 509), 'updatedExisting': True}, acknowledged=True)

In [33]:
from pymongo import UpdateOne

requests = []

for i in range(len(ids)):
    # create an UpdateOne request for each document
    requests.append(UpdateOne(
        {"_id": ids[i]},
        {"$addToSet": {"DenseVectorList": "all-mpnet-base-v2"},
         "$set": {"DenseVector_all-mpnet-base-v2": embeddings2[i].tolist()}}
    ))

# perform the bulk update
collection.bulk_write(requests)


.................................................

BulkWriteResult({'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 484, 'nModified': 484, 'nRemoved': 0, 'upserted': []}, acknowledged=True)

In [30]:
# Now that I have the embeddings I need to store them in the database
# I will modify the existing collection, inserting a string "all-mpnet-base-v2" in the
# property DenseVectorList avoiding duplicates then store the corresponding embeddings
# in property DenseVector_all-mpnet-base-v2

# start cyclying the documents
for i in range(len(ids)):
    # perform the update in place using the $addToSet operator
    # https://docs.mongodb.com/manual/reference/operator/update/addToSet/
    # also populate the property DenseVector_all-mpnet-base-v2 with the corresponding
    # embedding
    collection.update_one(
        {"_id": ids[i]},
        {"$addToSet": {"DenseVectorList": "all-mpnet-base-v2"},
         "$set": {"DenseVector_all-mpnet-base-v2": embeddings2[i].tolist()}}
    )
    #each 10 documents print a dot
    if i % 10 == 0:
        print(".", end="")



TypeError: 'Collection' object is not callable. If you meant to call the 'initialize_unordered_bulk_op' method on a 'Collection' object it is failing because no such method exists.